Esta notebook sirve para verificar en los cómputos:
- Que no haya campos vacíos
- Que todos los valores sean enteros
- Que no haya caracteres de espacio dentro del valor de la celda
- Saber cuál es el valor máximo de municipios y distritos y guardarlo en una variable
- Que los valores del conjunto de distritos y municipios sean una serie ininterrumpida que va de 1 (o 0) al máximo de municipios o distritos de esa elección



# Sección Observar tipos de dato en análisis

In [ ]:
import pandas as pd
from IPython.display import display, HTML

content_books = !ls *.xlsx | tr '\n' '\n'
sheets=['Election','Sources','Candidates','Coalitions','Election_counts']
for sheet in sheets:
  for file_name in content_books:
    print('\n-----------------------------------------')
    print(file_name)
    print('-----------------------------------------')
    sheet_to_df = pd.read_excel(file_name, sheet_name=sheet)
    #for sheet,data in sheet_to_df_map.items():
    #print('\t*****--*****--*****')
    print('\tHoja:',sheet)
    if sheet_to_df.isnull().sum().sum() >0:
      print(f'\n\t.   ∞ En total tiene {sheet_to_df.isnull().sum().sum()} NaN  ∞   . \n')
      null_columns = sheet_to_df.columns[sheet_to_df.isnull().any()]
      print('\t\t Conteo por columna:\n') 
      msg=''.join(sheet_to_df[null_columns].isnull().sum().to_string()).split('\n')
      print('\t\t',',\n\t\t '.join(msg))

    print('\t**_**--**_**--**_**--**_**--**_**\n')
    #my_data=pd.DataFrame(data)
    s1 = list(sheet_to_df.columns)
    s2 = list(sheet_to_df.dtypes)
    
    df_types = pd.DataFrame([s2], columns=s1)
    display(HTML(df_types.to_html()))
    print()
    display(HTML(sheet_to_df.head(2).to_html()))


-----------------------------------------
estado-de-mexico_municipios_2015.xlsx
-----------------------------------------
	Hoja: Election
	**_**--**_**--**_**--**_**--**_**



,State,Year,Election_type,Total_votes,Nominal_list,Computed_transactions,Fidelity
0,int64,int64,object,int64,int64,int64,int64


,State,Year,Election_type,Total_votes,Nominal_list,Computed_transactions,Fidelity
0,15,2015,Municipios,5554072,11023636,0,3



-----------------------------------------
estado-de-mexico_municipios_2015.xlsx
-----------------------------------------
	Hoja: Sources

	.   ∞ En total tiene 2 NaN  ∞   . 

		 Conteo por columna:

		 File              1,
		 Considerations    1
	**_**--**_**--**_**--**_**--**_**



,Name,Source,File,Fields,Considerations
0,object,object,float64,object,float64


...

,ID_Election_count,Coalition,Name,Players
0,int64,object,object,object


,ID_Election_count,Coalition,Name,Players
0,2,"PRI,PVEM,PANAL",De la mano con la gente,"PRI,PVEM,PANAL"
1,4,"PRI,PVEM,PANAL",De la mano con la gente,"PRI,PVEM,PANAL"



-----------------------------------------
estado-de-mexico_municipios_2015.xlsx
-----------------------------------------
	Hoja: Election_counts
	**_**--**_**--**_**--**_**--**_**



,ID,Municipality/District/State,...,....1,....2,Not_reg,Null_votes
0,object,object,object,object,object,object,object


,ID,Municipality/District/State,...,....1,....2,Not_reg,Null_votes


# Sección separar independientes en polling places

In [ ]:
import pandas as pd
from IPython.display import display, HTML
content_books = !ls *.xlsx | tr '\n' '\n'

In [ ]:
def some_column_begin_with(columns, phrase='C. Independiente'):
  for column in columns:
    if phrase in column:
      return True
  return False

content_books = !ls *.xlsx | tr '\n' '\n'

In [ ]:
new_computes=[]
computes_mun=[]
for n in [1]:
  for file_name in content_books:
    print('----------------......----.....')
    print(file_name)
    print('----------------......----.....')
    sheet_to_df_map = pd.read_excel(file_name, sheet_name=None)
    for sheet,data in sheet_to_df_map.items():
      if sheet == 'Polling_places':
        print()
        print(sheet)
        print()
        my_data = pd.DataFrame(data)
        if some_column_begin_with(my_data.columns, 
                                  phrase='C. Independiente'):
          muns=my_data['municipality_id'].unique()
          print(muns)
          for mun in list(muns):
            df_mun = my_data.loc[my_data['municipality_id'] == mun]
            #n_independiente='C. Independiente 1'
            n_independientes=['C. Independiente 1', 'C. Independiente 2','C. Independiente 3']
            for n_independiente in n_independientes:
              #if n_independiente in list(df_mun.columns):
              suma_ind_n = df_mun[n_independiente].sum()
              if suma_ind_n>0:
                df_mun[n_independiente +'_'+str(mun)] = df_mun[n_independiente]
                #display(HTML(df_mun.head(5).to_html()))
              computes_mun=df_mun.to_dict('records')
              new_computes.extend(computes_mun)


----------------......----.....
chihuahua_municipios_2016_polling_place.xlsx
----------------......----.....

Polling_places

[ 1  3  9 14 25 33 47 52 53 56 57 64  4  8 12 18 27 42 48 51 54  2  5  6
  7 10 11 13 15 16 17 19 20 21 22 23 24 26 28 29 30 31 32 34 35 36 37 38
 39 40 41 43 44 45 46 49 50 55 58 59 60 61 62 63 65 66 67]


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [ ]:
from openpyxl import load_workbook
def add_new_sheet(name_book,df,name_sheet):
  ''' add new sheet, if the sheet exists it will be overwritten'''
  book = load_workbook(name_book)
  writer = pd.ExcelWriter(name_book, engine = 'openpyxl', mode='w')
  writer.book = book
  writer.sheets = dict((ws.title, ws) for ws in book.worksheets) 
  #print(writer.sheets) 
  df.to_excel(writer, sheet_name = name_sheet, index=False)
  writer.save()
  writer.close()

In [ ]:
new_data=pd.DataFrame(new_computes)
new_data=new_data.fillna(0)
new_data=new_data.astype(int)

new_data.to_excel('chihuahua_municipios_2016_polling_place_split.xlsx')
add_new_sheet('chihuahua_municipios_2016_polling_place.xlsx', new_data, 'Polling_places_split_ind'  )

## SECCIÓN 1: DEFINIR BIBLIOTECAS Y ARCHIVOS

In [ ]:
#@title Importamos bibliotecas
import pandas as pd
import numpy as np
from openpyxl import load_workbook

In [ ]:
#@title Definir funciones a usar
def add_new_sheet_index(name_book,df,name_sheet):
  ''' add new sheet, if the sheet exists new sheet with other index will be created'''
  book = load_workbook(name_book)
  writer = pd.ExcelWriter(name_book, engine = 'openpyxl', mode='w')
  writer.book = book
  df.to_excel(writer, sheet_name = name_sheet, index=False)
  writer.save()
  writer.close()

def add_new_sheet(name_book,df,name_sheet):
  ''' add new sheet, if the sheet exists it will be overwritten'''
  book = load_workbook(name_book)
  writer = pd.ExcelWriter(name_book, engine = 'openpyxl', mode='w')
  writer.book = book
  writer.sheets = dict((ws.title, ws) for ws in book.worksheets) 
  #print(writer.sheets) 
  df.to_excel(writer, sheet_name = name_sheet, index=False)
  writer.save()
  writer.close()

def types_columns_unique(types_dataframe):
  ''' generate a set of types columns from columns dataframe'''
  return ','.join(map(str, set(list(types_dataframe))))


def adjust_integers(content_books, sheet_name, new_sheet_name, regex_pattern_aditional=r'^-*$|^o+$|^O+$'):
  ''' Try to convert all columns in a sheet_name to integer, 
      from each book in content_books '''
  all_columns_integer={}
  print(f'\nSe creará NUEVA HOJA "{new_sheet_name}" para cada uno de los archivos siguientes, con los ajustes que se muestran en summary')
  for book in content_books:
    sustituir_vacios = True
    original_all_numbers = False
    all_columns_integer[book]={}
    print('\n\nLeyendo Libro:',book)
    df_temp = pd.read_excel(book,sheet_name=sheet_name)
    try:
      df_temp = df_temp.astype(int, errors='raise')
      sustituir_vacios = False
      original_all_numbers = True
    except ValueError as e:
      print('\n\tAVISO: Se encontraron valores que no se pueden convertir a entero directamente...')
      #print('\n\t\t!Info error 1:',e)
      all_columns_integer[book]['integer'] = False
    if sustituir_vacios:
      try:
        df_temp=df_temp.replace(r'^\s*$', 0, regex=True).replace(regex_pattern_aditional, 0, regex=True)
        #print(f'\n\tTRATAREMOS de SUSTITUIR valores VACÍOS, y los que cumplan este PATRÓN: {regex_pattern_aditional} por ceros')
      except ValueError as e:
        print('\n\tAVISO: Algún valor NO SE PUDO SUSTITUIR por 0\'s \:\( ')
        #print ('\n\t\t!Info error 2:',e)
    df_temp=df_temp.fillna(0)

    try:
      df_temp = df_temp.astype(int, errors='raise')
      all_columns_integer[book]['integer'] = True
    except ValueError as e:
      print('\n\tAVISO: Existe algún valor que no se puede convertir a entero :S')
      #print('\n\t\t!Info error 3', e)
      all_columns_integer[book]['integer'] = False
    
    all_columns_integer[book]['original_all_numbers'] = original_all_numbers
    all_columns_integer[book]['types_unique'] = types_columns_unique(df_temp.dtypes)

    g = dict(df_temp.columns.to_series().groupby(df_temp.dtypes).groups)   
    types_columns={}

    for k,v in g.items():
      if not 'int' in (str(k)):
        types_columns[str(k)]=','.join(list(v))

    if not all_columns_integer[book]['integer']:
      print('\n\tAVISO: Intenta un proceso individual para cada columna ...')
    types_columns.update({'book':book.split('.')[0]})
    all_columns_integer[book]['types_by_column'] = types_columns
    add_new_sheet(name_book=book,df=df_temp,name_sheet=new_sheet_name)


  print('\nFin del proceso :)')
  return all_columns_integer


In [ ]:
#@title Leemos cuales son los archivos con terminación xlsx
content_books = !ls *.xlsx | tr '\n' '\n'

## SECCIÓN 2: CONVERSIÓN ENTEROS 
######  

*   Modifica a integer los valores de las columnas
*   De no ser posible la conversión, intenta reemplazar a 0's espacios en blanco y los que cumplan con un patrón regular.
*   En el mismo archivo agrega una hoja nueva 'Polling_places_corrections'

In [ ]:
#@title Column to Integer Conversion Proccess
all_columns_integer = adjust_integers(content_books = content_books, 
                                    sheet_name = 'Polling_places', 
                                    new_sheet_name = 'Polling_places_corrections',
                                    regex_pattern_aditional = r'^-*$|^o+$|^O+$|^/+$')



Se creará NUEVA HOJA "Polling_places_corrections" para cada uno de los archivos siguientes, con los ajustes que se muestran en summary


Leyendo Libro: aguascalientes_diputados_2013_polling_place.xlsx


Leyendo Libro: baja-california_municipios_2013_polling_place (4).xlsx


Leyendo Libro: baja-california_municipios_2016_polling_place.xlsx

Fin del proceso :)


In [ ]:
#@title Summary Conversion Integers
df = pd.DataFrame([[key.split('.')[0],value['integer'],value['original_all_numbers'], value['types_unique']] for key,value in all_columns_integer.items()], columns=['name_book', 'Finally Integer?','Began with numbers?', 'Types unique'])
df

,name_book,Finally Integer?,Began with numbers?,Types unique
0,aguascalientes_diputados_2013_polling_place,True,True,int64
1,baja-california_municipios_2013_polling_place (4),True,True,int64
2,baja-california_municipios_2016_polling_place,True,True,int64


## Sección Suggest changes in columns cells

In [ ]:
#@title Summary Columns cannot convert to Integers in a book
columnas_not_int=pd.DataFrame([v['types_by_column'] for v in all_columns_integer.values() if not v['integer']])
pd.set_option('display.max_colwidth', 0)
columnas_not_int


""


In [ ]:
#@title Proccess to suggest changes in columns
def fix_multiple_types(n): 
    import re
    tipo=str(type(n))
    x=n
    if 'int' in tipo:
      return n
    if 'str' in tipo:
      numbers_patron = re.compile(r"[\d]", re.X)
      number=''.join(numbers_patron.findall(n))
      try: 
        n=int(number)
        print( '\t\t\tAhora: ',str(type(n)), str(n), '\tAntes:', tipo, str(x))
        return n
      except:
        pass
    elif 'float' in tipo:
      if n >= 0:
        n=int(n)
        return n
    print('\t\t\tAhora:', str(type(0)),str(0), '\tAntes:', str(type(x)), str(x))
    return 0

def fix_multiple_types_suggest_columns(n): 
    import re
    tipo=str(type(n))
    x=n
    if 'int' in tipo:
      return n
    if 'str' in tipo:
      numbers_patron = re.compile(r"[\d]", re.X)
      number=''.join(numbers_patron.findall(n))
      try: 
        n=int(number)
        n=''.join(['Ahora: ',str(type(n)),' ', str(n), ' Antes: ', tipo, str(x)])
        return n
      except:
        pass
    elif 'float' in tipo:
      if n >= 0:
        n=int(n)
        return n
    return ''.join(['Ahora: ', str(type(0)),' ', str(0), ' Antes: ', str(type(x)), str(x)])
   

for k,v in all_columns_integer.items():
  df_temp_columns_suggest = pd.read_excel(k,sheet_name='Polling_places')
  df_temp_sheet_suggest = pd.read_excel(k,sheet_name='Polling_places')

  print('LIBRO:',k,'\t')
 
  if not v['integer']:
    for key, tipos in v['types_by_column'].items():
      if key == 'book':
        continue
      print('TIPO:',key)
      
      for c in tipos.split(','):
        print('\n\tColumna:\n\t',c,'\n')
        df_temp_sheet_suggest[c]=list(map(fix_multiple_types,list(df_temp_sheet_suggest[c])))
        #df_temp_columns_suggest[c+'suggested']= df_temp_columns_suggest[c].map(fix_multiple_types_suggest_columns)
        df_temp_columns_suggest[c]= df_temp_columns_suggest[c].map(fix_multiple_types_suggest_columns)

    df_temp_sheet_suggest.astype(int, errors='ignore')
    df_temp_columns_suggest.astype(int, errors='ignore')
    

    add_new_sheet(name_book=k,df=df_temp_sheet_suggest,name_sheet='Polling_places_corrections')
    add_new_sheet(name_book=k,df=df_temp_columns_suggest,name_sheet='Polling_places_suggested')


LIBRO: aguascalientes_diputados_2013_polling_place.xlsx 	
LIBRO: baja-california_municipios_2013_polling_place (4).xlsx 	
LIBRO: baja-california_municipios_2016_polling_place.xlsx 	


## SECCION VER HOJA ELECCION

In [ ]:
#@title Comprobar hoja election
columnas_plantilla_election1=['State','Year','Election_type','Total_votes','Nominal_list','Computed_transactions','Fidelity']
#columnas_plantilla_election=['State','Year','Election_type','Total_votes','Nominal_list','Computed_transactions','Fidelity']
#columnas_plantilla_Polling_places=['section_id', 'local_disctrict_id', 'municipality_id', 'state_id','Not_reg','Null_votes']
columnas_plantilla_election=set(['State','Year','Election_type','Total_votes','Nominal_list','Computed_transactions','Fidelity'])
lista_elections=[]
dicc_election={}

for k in content_books:
  print(k)
  xls = pd.ExcelFile(k)
  if  not 'Election' in xls.sheet_names:
    print('\n\tNo existe Election')
    continue


  data_election=pd.read_excel(k,sheet_name='Election')
  

  dicc_election['book']=k
  columnas_hoja_election=set(data_election.columns)

  if  not columnas_plantilla_election-columnas_hoja_election:
    print('\n\tColumnas igual a la plantilla!')
    header_equal=True
  else:
    print('Falta al menos una columna con respecto a la plantilla')
    header_equal=False

  dicc_election['header_equal']=header_equal

  dicc_election['types_column']=list(data_election.dtypes)
  dicc_election['values_fila']=list(data_election.iloc[0])
  lista_elections.append(dicc_election)
  dicc_election['total_values']=len(data_election.iloc[0])
  dicc_election['total_header']=len(data_election.iloc[0])





aguascalientes_diputados_2013_polling_place.xlsx

	Columnas igual a la plantilla!
baja-california_municipios_2013_polling_place (4).xlsx

	Columnas igual a la plantilla!
baja-california_municipios_2016_polling_place.xlsx

	Columnas igual a la plantilla!


In [ ]:
dicc_election

{'book': 'baja-california_municipios_2016_polling_place.xlsx',
 'header_equal': True,
 'total_header': 7,
 'total_values': 7,
 'types_column': [dtype('int64'),
  dtype('int64'),
  dtype('O'),
  dtype('int64'),
  dtype('int64'),
  dtype('int64'),
  dtype('int64')],
 'values_fila': [2, 2016, 'Municipios', 833160, 2570078, 0, 3]}

In [ ]:
columnas_plantilla_election1

['State',
 'Year',
 'Election_type',
 'Total_votes',
 'Nominal_list',
 'Computed_transactions',
 'Fidelity']

In [ ]:
#@title Election summary
df_=pd.DataFrame([dicc_election['types_column'],dicc_election['values_fila'] ], index=['types','values'],columns=columnas_plantilla_election1)
df_

,State,Year,Election_type,Total_votes,Nominal_list,Computed_transactions,Fidelity
types,int64,int64,object,int64,int64,int64,int64
values,2,2016,Municipios,833160,2570078,0,3


## SECCIÓN 3: EDICIÓN DE NOMBRES DE COLUMNAS

In [ ]:
#@title Definir funciones a usar
def reemplazar_nombres_columnas(df_temp, reemplazar):
  '''Recibe un nuevo nombre de columna y lo reemplaza por una anteriormente dado'''
  
  if reemplazar == 'Y':

    print("\t¿Cuál es nombre de columna a reemplazar? ", end="")
    columna_anterior = input()
    print("\t¿Cuál es el nuevo nombre de columna?", end="")
    columna_nueva = input()
    print(f"\n\tSe reemplazará: {columna_anterior} por {columna_nueva}")

    if columna_anterior in df_temp.columns and columna_nueva:
  
      df_temp.rename(columns={ columna_anterior :columna_nueva},inplace=True)
      
      print ('\n\tREEMPLAZO LISTO!\n')
      print('\n\tESTAS SON LAS COLUMNAS AHORA:\n\n\t',df_temp.columns)
    else:
      print(f'{columna_anterior} No es una columna existente! y/o recuerda agregar el texto por el cual sustituir')

  print('\n\t ¿Deseas reemplazar el nombre de otra columna? (Y/N)')
  reemplazar = input()

  if reemplazar == 'Y':
    df_temp=reemplazar_nombres_columnas(df_temp, reemplazar)
    print('\tSe actualizará el nombre de la columna en el archivo de excel')
  else:
    print('\tFinalizando edición de columnas de este archivo...')

  return df_temp


def rename_columns(content_books=[], sheet_name='Polling_places_corrections'):
  '''Reads the name and renames certain columns of a sheet'''
  for book in content_books:

    print('\n\nNOMBRE DEL ARCHIVO:',book)
    df_temp=pd.read_excel(book,sheet_name=sheet_name)
    print('\n\n\tESTAS SON LAS COLUMNAS PRESENTES:\n\n')
    print('\t',df_temp.columns)
    print('\n\t¿Deseas reemplazar el nombre de alguna columna? (Y/N)')

    reemplazar = input()

    if reemplazar == 'Y': 
      df_temp = reemplazar_nombres_columnas(df_temp,reemplazar)
      add_new_sheet(name_book=book,df=df_temp,name_sheet=sheet_name)
      print('\tSe actualizó el nombre de la columna en el archivo de excel')
    else:
      if reemplazar == 'N':
        print('\tBye!')
      else:
        print('\tOpción no válida, pasaremos al siguiente archivo...')

  return print('\nProceso finalizado :D')

In [ ]:
#@title Proccess to help us to rename columns
rename_columns(content_books=content_books, sheet_name='Polling_places_corrections')



NOMBRE DEL ARCHIVO: aguascalientes_diputados_2013_polling_place.xlsx


	ESTAS SON LAS COLUMNAS PRESENTES:


	 Index(['section_id', 'local_district_id', 'municipality_id', 'state_id', 'PAN',
       'PRI', 'PRD', 'PT', 'PVEM', 'MC', 'PANAL', 'PAN,PRD', 'PRI,PVEM',
       'Not_reg', 'Null_votes'],
      dtype='object')

	¿Deseas reemplazar el nombre de alguna columna? (Y/N)
N
	Bye!


NOMBRE DEL ARCHIVO: baja-california_municipios_2013_polling_place (4).xlsx


	ESTAS SON LAS COLUMNAS PRESENTES:


	 Index(['section_id', 'local_district_id', 'municipality_id', 'state_id', 'MC',
       'PAN,PRD,PANAL,PBC', 'PRI,PVEM,PES,PT', 'Not_reg', 'Null_votes'],
      dtype='object')

	¿Deseas reemplazar el nombre de alguna columna? (Y/N)
N
	Bye!


NOMBRE DEL ARCHIVO: baja-california_municipios_2016_polling_place.xlsx


	ESTAS SON LAS COLUMNAS PRESENTES:


	 Index(['section_id', 'local_district_id', 'municipality_id', 'state_id', 'PAN',
       'PRI', 'PRD', 'PT', 'PH', 'PVEM', 'PBC', 'PANAL', 'PES', 'M

## SECCIÓN 4: EVALUACIÓN CONTINUIDAD DISTRITOS

In [ ]:
#@title Definir funciones a usar
def continuidad_columna(content_books=[], 
                        sheet_name='Polling_places_corrections',
                        column_to_evaluate='local_district_id'
                        ):
  ''' Evaluá si existe continuidad en columma determinada de una hoja del libro dada, compromando que sea vacía la diferencia simétrica'''
  evaluacion_distritos={}

  print ('\nPara todos los archivos siguientes se evaluará la hoja:',sheet_name,'y la columna:' ,column_to_evaluate)
  for book in content_books:

    print('\nLibro:',book)
    evaluacion_distritos[book]={}

    df_temp = pd.read_excel(book,sheet_name=sheet_name)

    _df=df_temp.agg({column_to_evaluate:['max','min']})
    #print('MIN Y MAX de DISTRITOS:\n\n',_df)
    _unicos = df_temp[column_to_evaluate].unique()
    #print(f'\n\tValores UNICOS presentes en la columna {column_to_evaluate}:\n\n\t',_unicos)

    evaluacion_distritos[book]['min'] = _df[column_to_evaluate]['min']
    evaluacion_distritos[book]['max'] = _df[column_to_evaluate]['max']
    #evaluacion_distritos[book]['count'] = _df[column_to_evaluate]['count']
    evaluacion_distritos[book]['shape']= df_temp.shape
    evaluacion_distritos[book]['size']= df_temp.size

    a = set(_unicos)
    b = set(np.arange(evaluacion_distritos[book]['min'],evaluacion_distritos[book]['max'] + 1))
    c = a^b

    if c:
      print(f'\n\tAviso! Conjunto de ID DIFERENTES entre Lista Generada de Continuos y la columna {column_to_evaluate} :\n\n\t', c)
    else:
      print(f'\tWuju! Los {column_to_evaluate} archivo son continuos!\n')
    
    evaluacion_distritos[book]['diferencia']=", ".join(map(str,c))
  return evaluacion_distritos

In [ ]:
#@title Process to evaluate continuity in a column
evaluacion_distritos=continuidad_columna(content_books=content_books, 
                        sheet_name='Polling_places_corrections',
                        column_to_evaluate='municipality_id'
                        )


Para todos los archivos siguientes se evaluará la hoja: Polling_places_corrections y la columna: municipality_id

Libro: aguascalientes_diputados_2013_polling_place.xlsx
	Wuju! Los municipality_id archivo son continuos!



In [ ]:
#@title Differences found in column continuity

df = pd.DataFrame([[key.split('.')[0],value['diferencia'],value['min'], value['max'], value['shape'], value['size']] for key,value in evaluacion_distritos.items()], 
                  columns=['Libro', 'IDs No Continuos', 'Mínimo', 'Máximo', 'Filas,Columnas', 'Total Celdas'])
df

,Libro,IDs No Continuos,Mínimo,Máximo,"Filas,Columnas",Total Celdas
0,aguascalientes_diputados_2013_polling_place,,0,0,"(1421, 15)",21315


## SECCIÓN 5: GUARDAR EN NUESTRO DRIVE

In [ ]:
#@title Autenticarse en google drive
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:
#@title Crear carpeta en drive a donde se copiaran todos los archivos xlsx de esta sesión de notebook
name_folder = "aguascalientes_test9" #@param {type:"string"}

if name_folder:
  try: 
    !mkdir  'drive/My Drive/'{name_folder}
    !cp /content/*.xlsx 'drive/My Drive/'{name_folder}
    print(f'Listo! Archivos guardados en tu carpeta de drive: {name_folder} :D')
  except:
    print('Agrega un nombre para la carpeta')
  

mkdir: cannot create directory ‘drive/My Drive/aguascalientes_test9’: File exists
Listo! Archivos guardados en tu carpeta de drive: aguascalientes_test9 :D


In [ ]:
archivos= ! ls *.xlsx | tr '\n' '\n'
from google.colab import files

for archivo in archivos:
  files.download(archivo)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>